In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from tqdm.notebook import tqdm

from spell_checking.baselines import sec
from spell_checking import BENCHMARK_DIR, DATA_DIR, CONFIG_DIR

import nsc
from nsc import BeamSearch
from nsc.api import utils
from nsc.data.utils import clean_sequence

In [10]:
bert = sec.SECNeuspellBaseline("bert", force_cpu=True)

data folder is set to `/home/sebastian/anaconda3/envs/masters_thesis/lib/python3.8/site-packages/neuspell/../data` script
loading vocab from path:/home/sebastian/anaconda3/envs/masters_thesis/lib/python3.8/site-packages/neuspell/../data/checkpoints/subwordbert-probwordnoise/vocab.pkl
initializing model


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


SubwordBert(
  (bert_dropout): Dropout(p=0.2, inplace=False)
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): 

In [3]:
os.environ["NSC_CONFIG_DIR"] = CONFIG_DIR
os.environ["NSC_DATA_DIR"] = DATA_DIR

In [4]:
sec_words = nsc.SpellingErrorCorrector.from_experiment("../../experiments/SEC_WORDS_NMT/\
transformer_sec_words_nmt_wikidump_paragraphs_sed_words_and_sec_bookcorpus_paragraphs_sed_words_and_sec_25_03_2022_16_58_40/")

2022-05-17 12:26:24,251 [SPELLING_ERROR_CORRECTION] [INFO] [35811] running spelling error correction on device NVIDIA GeForce GTX 1080 Ti (11,178MiB memory, 6.1 compute capability, 28 multiprocessors)


#### hyphenation

In [8]:
sec_words.correct_text("He does not like his brotherin-law!")

'He does not like his brother-in-law!'

In [11]:
bert.inference(["He does not like his brotherin-law!"])[0]

'He does not like his brotherin-law!'

#### technical terms

In [9]:
sec_words.correct_text("The endoplasimc reticulm is part of a cell.")

'The endoplasmic reticulum is part of a cell.'

In [12]:
bert.inference(["The endoplasimc reticulm is part of a cell."])[0]

'The endoplasimc reticulm is part of a cell.'

#### apostrophe

In [10]:
sec_words.correct_text("He lives in his uncle' house.")

"He lives in his uncle's house."

In [13]:
bert.inference(["He lives in his uncle' house."])[0]

"He lives in his uncles' house."

#### context

In [11]:
sec_words.correct_text("Their is no place for me.")

[['Their is no place for me.']]


'Their is no place for me.'

In [14]:
sec_words.correct_text("They was playing football.")

[['They was playing football.']]


'They was playing football.'

In [13]:
bert.inference(["Their is no place for me."])[0]

'There is no place for me.'

In [15]:
bert.inference(["They was playing football."])[0]

'They were playing football.'